In [20]:
import os

import numpy as np
import pandas as pd
path = 'DAT-WS2425-Projektarbeit\DAT-WS2425-Projektarbeit\Daten\Strompreisdaten'
extension = '.csv'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
files = [file for file in os.listdir(path) if file.endswith(extension)]

dfs = []
filecounter =0
for file in files:
    df = pd.read_csv(os.path.join(path, file))
    dfs.append(df)
    filecounter += 1
    
print("total files: ", filecounter)
df = pd.concat(dfs, ignore_index=True)


total files:  258


In [21]:
df['Datum (MEZ)'] = pd.to_datetime(df['Datum (MEZ)'])
df['Datum (MESZ)'] = pd.to_datetime(df['Datum (MESZ)'])
print(len(df['Datum (MESZ)']))

43392


In [22]:
df = df.drop(df[df['Datum (MEZ)']<'2020-01-01'].index)

In [23]:
# Separating and Creating new columns for Date and Time from Datum (MEZ) and Datum (MESZ)
df['Datum'] = df['Datum (MEZ)'].dt.date
df['Zeit'] = df['Datum (MEZ)'].dt.time
df['Datum'] =df['Datum'].fillna(df['Datum (MESZ)'].dt.date)
df['Zeit'] =df['Zeit'].fillna(df['Datum (MESZ)'].dt.time)
cols = df.columns.tolist()
print(cols)

['Datum (MEZ)', 'Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Day Ahead Auktion Preis (EUR/MWh)', 'Datum (MESZ)', 'Datum', 'Zeit']


In [24]:
cols = ['Datum', 'Zeit','Datum (MEZ)', 'Leistung nicht erneuerbar (MW)', 'Leistung erneuerbar (MW)', 'Day Ahead Auktion Preis (EUR/MWh)', 'Datum (MESZ)']
df = df[cols]
df = df.drop(columns=['Datum (MEZ)', 'Datum (MESZ)'])
df = df.reset_index(drop=True)
df

,Datum,Zeit,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
0,2020-01-01,00:00:00,18754.70,13961.99,41.88
1,2020-01-01,01:00:00,18337.54,14242.17,38.60
2,2020-01-01,02:00:00,18122.83,14584.20,36.55
3,2020-01-01,03:00:00,18323.93,14746.24,32.32
4,2020-01-01,04:00:00,18327.59,14924.48,30.85
...,...,...,...,...,...
43339,2024-12-10,19:00:00,NaN,NaN,180.17
43340,2024-12-10,20:00:00,NaN,NaN,166.60
43341,2024-12-10,21:00:00,NaN,NaN,151.15
43342,2024-12-10,22:00:00,NaN,NaN,137.94


In [26]:
# New df_daily with sum of Leistung and average of Auktion preis for every day
df_daily = df.groupby(['Datum']).agg({'Leistung nicht erneuerbar (MW)': 'sum', 'Leistung erneuerbar (MW)': 'sum', 'Day Ahead Auktion Preis (EUR/MWh)' : 'mean'})
df_daily

,Leistung nicht erneuerbar (MW),Leistung erneuerbar (MW),Day Ahead Auktion Preis (EUR/MWh)
Datum,,,
2020-01-01,468837.03,441093.57,34.964167
2020-01-02,505002.46,605620.76,38.869167
2020-01-03,427798.33,968577.88,26.111250
2020-01-04,411321.40,987041.46,20.322917
2020-01-05,508187.80,514534.00,35.552500
...,...,...,...
2024-12-06,482833.73,969065.85,86.374583
2024-12-07,386654.54,834428.48,71.508333
2024-12-08,455016.51,609461.18,99.068333
